# Парсинг постов и комментариев с групп ВК

## Step by step

Импорт необходимых модулей и VKParser

In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.data_getter import VKParser

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sashamorozov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Ввод данных о группе, с которой будут парситься посты и комментарии:
* domain – указание на нужную группу ВК. Идет после последнего слэша в ссылке (в примере это mytndvor 'https://vk.com/mytndvor')
* token – ключ от приложения ВК, которое нужно создать для загрузки данных

In [13]:
domain = 'mytndvor' # группа "центральный район за комфортную среду обитания"
token = '...'

In [14]:
VKParser.get_owner_id_by_domain(domain, access_token=token)

-129354225

Запуск парсера постов, на вход:
* domain
* token 
* кол-во постов, получаемых при 1 запросе
* дата, до которой скачиваются данные в формате гггг.мм.дд.

In [5]:
posts_df = VKParser().run_posts(domain, token, 100, "2024-03-01")

2024-02-17 10:30:01
posts downloaded


3 первых полученных поста из итогового датафрема

In [6]:
posts_df.head(3)

,date,id,text,views.count,likes.count,reposts.count,type,link
0,2024-04-05 19:30:00,337309,В рамках муниципальной предвыборной кампании 2...,3573,68,10,post,https://vk.com/@mytndvor-programma2024|предвыб...
1,2024-04-05 10:30:00,337154,«300 спартанцев» для инициализации референдума...,2984,131,24,post,https://vk.com/wall-129354225_335728|провести
2,2024-04-04 19:30:00,337038,На Тульской улице у дома 1 заработал киоск по ...,6315,216,37,post,NaN


Запуск парсера комментариев, на вход:
* domain
* id постов из прошлого датафрейма
* token

In [7]:
post_ids = posts_df["id"].to_list()
comments_df = VKParser().run_comments(domain, post_ids=post_ids, access_token=token)

100%|██████████| 100/100 [10:35<00:00,  6.35s/it]

comments downloaded


3 первых полученных комментария из итогового датафрема

In [8]:
comments_df.head(3)

,id,date,text,post_id,parents_stack,likes.count,type
0,337312,2024-04-05 16:41:40,"Замечательно, что есть неравнодушные люди, кот...",337309,[],12,comment
1,337334,2024-04-05 18:35:48,"[id7374803|Любовь], помогать словом , это прек...",337309,[337312],1,comment
2,337335,2024-04-05 18:38:54,"[id342126|Ирина], иногда людям достаточно подс...",337309,[337312],5,comment


Запуск парсера постов и комментариев с объединением в один датафрейм, на вход:
* domain
* token 
* кол-во постов/комментариев, получаемых при 1 запросе
* дата, до которой скачиваются данные в формате гггг.мм.дд.

In [9]:
result_df = VKParser().run_parser(domain, token, step=100, cutoff_date='2024-03-20') 

2024-02-17 10:30:01
posts downloaded


100%|██████████| 100/100 [09:55<00:00,  5.96s/it]


comments downloaded


Пять случайных записей из итогового датафрейма

In [10]:
result_df.sample(5)

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack
1806,2024-03-27 19:16:55,335080,"[id4649323|Елена], фото 1 из Капеллы- концерт ...",NaN,1,NaN,reply,NaN,335054.0,335065
3708,2024-03-18 04:54:07,333025,"[id2983567|Андрей], законодательство плохо зна...",NaN,0,NaN,reply,NaN,332749.0,332759
1152,2024-04-03 09:44:35,336512,"[id2909910|Денис], Вам самим не смешно? Я? Опа...",NaN,1,NaN,reply,NaN,335728.0,335935
5276,2024-02-28 10:07:52,330929,👏🏻Молодцы👏🏻,NaN,3,NaN,comment,NaN,330906.0,None
2566,2024-03-22 17:46:54,334024,Полный запрет.,NaN,10,NaN,comment,NaN,333949.0,None


## Запуск парсера на несколько групп одновременно 

In [3]:
domain = ['my_krd', 'krd_23', 'kraskrd', 'typical_krd'] # домены групп в '...'  
token = '6c44eaff6c44eaff6c44eaffca6f52ec1466c446c44eaff09769584de7acabcd2ac1d26' # сервисный ключ от приложения ВК
for domain in domain:
    try:
        result_df = VKParser().run_parser(domain, token, step=100, cutoff_date='2023-01-01') # парсинг постов и комментариев в один датафрейм
        result_df.to_csv(f'/Users/sashamorozov/Documents/Text_НИРМА/data_2/{domain}.csv', index=False) # сохранение результата в файл csv
        print(f'Done for {domain}!')
    except Exception as e:
        print(f'Error processing {domain}: {e}. Moving to the next domain.')
        continue

2024-04-19 09:00:01
2024-05-25 08:00:02
2024-05-20 18:44:00
2024-05-15 21:20:53
2024-05-10 20:00:01
2024-05-06 16:00:02
2024-05-02 06:00:00
2024-04-27 14:00:01
2024-04-23 16:00:00
2024-04-19 09:47:16
2024-04-14 23:00:00
2024-04-10 11:00:01
2024-04-05 14:30:00
2024-04-01 10:47:21
2024-03-28 11:00:02
2024-03-24 16:30:00
2024-03-21 19:30:00
2024-03-17 17:00:02
2024-03-13 08:00:02
2024-03-08 09:00:06
2024-03-04 09:00:02
2024-02-28 16:00:03
2024-02-23 18:05:52
2024-02-18 17:03:48
2024-02-13 19:34:50
2024-02-08 17:00:02
2024-02-04 02:00:00
2024-01-30 09:55:22
2024-01-25 22:00:00
2024-01-21 16:26:23
2024-01-17 14:00:02
2024-01-13 04:00:00
2024-01-08 10:45:01
2024-01-03 09:00:01
2023-12-30 04:00:00
2023-12-25 04:00:00
2023-12-20 23:30:00
2023-12-16 10:00:01
2023-12-12 12:00:03
2023-12-07 19:30:00
2023-12-04 10:16:56
2023-11-29 21:15:00
2023-11-26 19:00:03
2023-11-22 13:44:15
2023-11-17 19:30:12
2023-11-13 10:00:02
2023-11-08 18:00:03
2023-11-04 06:00:00
2023-10-30 20:28:01
2023-10-26 04:00:00


 45%|████▌     | 4621/10250 [1:59:16<2:25:17,  1.55s/it] 


KeyboardInterrupt: 